In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading and Preprocessing data**

In [3]:
import pandas as pd
#from sklearn.preprocessing import StandardScaler

def load_and_preprocess_data(file_path, dataset_name):
    # Loading the data
    data = pd.read_csv(file_path,encoding='latin1')



    return data


# Loading and preprocessing each dataset

demographic_data = load_and_preprocess_data(
    '/content/drive/MyDrive/demographic.csv',
    dataset_name='demographic'
)

diet_data = load_and_preprocess_data(
    '/content/drive/MyDrive/diet.csv',
    dataset_name='diet'
)

examination_data = load_and_preprocess_data(
    '/content/drive/MyDrive/examination.csv',
    dataset_name='examination'
)

labs_data = load_and_preprocess_data(
    '/content/drive/MyDrive/labs.csv',
    dataset_name='labs'
)

medications_data = load_and_preprocess_data(
    '/content/drive/MyDrive/medications - medications.csv',
    dataset_name='medications'
)
labeled_data=load_and_preprocess_data(
    '/content/drive/MyDrive/Labeled_data - Sheet1 (1).csv',
    dataset_name='labeled_data'
)
demographic_data = demographic_data.iloc[:, :3]
diet_data = diet_data.iloc[:, :3]
examination_data = examination_data.iloc[:, :3]
labs_data = labs_data.iloc[:, :3]
medications_data = medications_data.iloc[:, :3]

# Merging the datasets based on common column
data = pd.merge(demographic_data, diet_data,how='left')
data = pd.merge(data, examination_data,how='left')
data = pd.merge(data, labs_data,how='left')
data = pd.merge(data, medications_data,how='left')
data = pd.merge(data, labeled_data,how='left')
data=data.dropna()
print(data)


        SEQN  SDDSRVYR  RIDSTATR        WTDRD1        WTDR2D  PEASCST1  \
0      73557         8         2  16888.327864  12930.890649       1.0   
1      73557         8         2  16888.327864  12930.890649       1.0   
2      73558         8         2  17932.143865  12684.148869       1.0   
3      73558         8         2  17932.143865  12684.148869       1.0   
4      73558         8         2  17932.143865  12684.148869       1.0   
...      ...       ...       ...           ...           ...       ...   
20182  83724         8         2  32115.849467  46169.498764       1.0   
20183  83724         8         2  32115.849467  46169.498764       1.0   
20184  83724         8         2  32115.849467  46169.498764       1.0   
20185  83724         8         2  32115.849467  46169.498764       1.0   
20186  83724         8         2  32115.849467  46169.498764       1.0   

       PEASCTM1  URXUMA  URXUMS  RXDUSE  RXDCOUNT  SCORE  
0         620.0     4.3     4.3       1       2.0   

**Prediction using CNN model**

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



# Defining input features
input_features = ['SDDSRVYR', 'RIDSTATR', 'WTDRD1', 'WTDR2D',
                  'PEASCST1', 'PEASCTM1', 'URXUMA', 'URXUMS',
                  'RXDUSE', 'RXDCOUNT']

# Splitting the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Preparing input data and labels
train_features = train_data[input_features]
train_labels = train_data['SCORE']

test_features = test_data[input_features]
test_labels = test_data['SCORE']

# Converting NumPy arrays to tensors
train_features = tf.convert_to_tensor(train_features.values, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels.values, dtype=tf.float32)

test_features = tf.convert_to_tensor(test_features.values, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels.values, dtype=tf.float32)

# Defining the 1D CNN model
input_layer = Input(shape=(len(input_features), 1))  # Input shape for 1D CNN
conv1d_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
maxpooling1d_layer = MaxPooling1D(pool_size=2)(conv1d_layer)
flatten_layer = Flatten()(maxpooling1d_layer)
dense_layer_1 = Dense(64, activation='relu')(flatten_layer)
output_layer = Dense(1, activation='linear')(dense_layer_1)  # Linear activation for regression task

model = Model(inputs=input_layer, outputs=output_layer)

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(train_features, train_labels, epochs=10, batch_size=32)

# Evaluating the model on test data
loss = model.evaluate(test_features, test_labels)
print(f"Test Loss (MSE): {loss}")

# Making predictions on new data
new_data = pd.DataFrame({
    'SDDSRVYR': [8],
    'RIDSTATR': [2],
    'WTDRD1': [16888.327864],
    'WTDR2D': [12930.890649],
    'PEASCST1': [1],
    'PEASCTM1': [620],
    'URXUMA': [4.3],
    'URXUMS': [4.3],
    'RXDUSE': [1],
    'RXDCOUNT': [2]
})

# Converting the new data to a tensor
new_data = tf.convert_to_tensor(new_data.values, dtype=tf.float32)

# Reshaping the input data to match the 1D CNN input shape
new_data = tf.reshape(new_data, shape=(1, len(input_features), 1))

predicted_scores = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_scores[0][0]}")


Epoch 1/10
282/282 [==============================] - 2s 3ms/step - loss: 445283.0000
Epoch 2/10
282/282 [==============================] - 1s 3ms/step - loss: 52643.1914
Epoch 3/10
282/282 [==============================] - 1s 3ms/step - loss: 323711.7500
Epoch 4/10
282/282 [==============================] - 1s 3ms/step - loss: 72850.2891
Epoch 5/10
282/282 [==============================] - 1s 5ms/step - loss: 1994.8624
Epoch 6/10
282/282 [==============================] - 1s 4ms/step - loss: 3417.0308
Epoch 7/10
282/282 [==============================] - 1s 4ms/step - loss: 112174.4141
Epoch 8/10
282/282 [==============================] - 1s 5ms/step - loss: 1916.6345
Epoch 9/10
282/282 [==============================] - 1s 2ms/step - loss: 1524.0609
Epoch 10/10
71/71 [==============================] - 0s 2ms/step - loss: 15803.0791
Test Loss (MSE): 15803.0791015625
1/1 [==============================] - 0s 74ms/step
Predicted Healthcare Score: 21.71817398071289


**Prediction using feed-forward neural network**

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Defining input features
input_features = ['SDDSRVYR', 'RIDSTATR', 'WTDRD1', 'WTDR2D',
                  'PEASCST1', 'PEASCTM1', 'URXUMA', 'URXUMS',
                  'RXDUSE', 'RXDCOUNT']

# Splitting the data into train and test sets

train_data, test_data = train_test_split(data, test_size=0.33, random_state=42)

# Preparing input data
train_features = train_data[input_features]
train_labels = train_data['SCORE']

train_features = tf.convert_to_tensor(train_features.values, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels.values, dtype=tf.float32)

# Defining the neural network model
input_layer = Input(shape=(len(input_features),))
dense_layer_1 = Dense(64, activation='relu')(input_layer)
dense_layer_2 = Dense(32, activation='linear')(dense_layer_1)
output_layer = Dense(1, activation='linear')(dense_layer_2)  # Linear activation for regression task

model = Model(inputs=input_layer, outputs=output_layer)

# Compiling the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Training the model
model.fit(train_features, train_labels, epochs=10, batch_size=32)

# Evaluating the model on test data
test_features = test_data[input_features]
test_labels = test_data['SCORE']

test_features = tf.convert_to_tensor(test_features.values, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels.values, dtype=tf.float32)

loss = model.evaluate(test_features, test_labels)
print(f"Test Loss: {loss}")

# Making predictions on new data
new_data = pd.DataFrame({
    'SDDSRVYR': [8],
    'RIDSTATR': [2],
    'WTDRD1': [16888.327864],
    'WTDR2D': [12930.890649],
    'PEASCST1': [1],
    'PEASCTM1': [620],
    'URXUMA': [4.3],
    'URXUMS': [4.3],
    'RXDUSE': [1],
    'RXDCOUNT': [2]
})
new_data = tf.convert_to_tensor(new_data.values, dtype=tf.float32)

predicted_scores = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_scores[0][0]}")


Epoch 1/10
236/236 [==============================] - 2s 5ms/step - loss: 1371.5807
Epoch 2/10
236/236 [==============================] - 1s 4ms/step - loss: 212.0736
Epoch 3/10
236/236 [==============================] - 0s 2ms/step - loss: 227.0077
Epoch 4/10
236/236 [==============================] - 0s 2ms/step - loss: 232.8371
Epoch 5/10
236/236 [==============================] - 0s 2ms/step - loss: 196.0447
Epoch 6/10
236/236 [==============================] - 0s 2ms/step - loss: 249.7493
Epoch 7/10
236/236 [==============================] - 1s 2ms/step - loss: 234.2869
Epoch 8/10
236/236 [==============================] - 1s 3ms/step - loss: 200.9560
Epoch 9/10
236/236 [==============================] - 0s 2ms/step - loss: 126.5842
Epoch 10/10
117/117 [==============================] - 0s 2ms/step - loss: 65.9123
Test Loss: 65.91228485107422


1/1 [==============================] - 0s 72ms/step
Predicted Healthcare Score: 19.607385635375977
